In [1]:

from rbot import Session
from rbot import Runner
from rbot import BinanceConfig
from rbot import BinanceMarket
from rbot import init_debug_log
from rbot import init_log
from rbot import OrderSide
from rbot import time_string
from rbot import OrderStatus


In [2]:
class MyAgent:
    def on_clock(self, session, clock):
        print("on_clock", time_string(clock))
        session.market_order(OrderSide.Sell, 0.01)
        session.market_order(OrderSide.Buy, 0.01)




In [3]:
market = BinanceMarket(BinanceConfig.TEST_BTCUSDT)

agent = MyAgent()
runner = Runner()



session = runner.back_test(market, agent, 3600, 0, 0)

session

on_clock 2023-08-30T00:00:00.000000
on_clock 2023-08-30T01:00:00.000000
on_clock 2023-08-30T02:00:00.000000
on_clock 2023-08-30T03:00:00.000000
on_clock 2023-08-30T04:00:00.000000
on_clock 2023-08-30T05:00:00.000000
on_clock 2023-08-30T06:00:00.000000
on_clock 2023-08-30T07:00:00.000000
on_clock 2023-08-30T08:00:00.000000
on_clock 2023-08-30T09:00:00.000000
on_clock 2023-08-30T10:00:00.000000
on_clock 2023-08-30T11:00:00.000000
on_clock 2023-08-30T12:00:00.000000
on_clock 2023-08-30T13:00:00.000000
on_clock 2023-08-30T14:00:00.000000
on_clock 2023-08-30T15:00:00.000000
on_clock 2023-08-30T16:00:00.000000
on_clock 2023-08-30T17:00:00.000000
on_clock 2023-08-30T18:00:00.000000
on_clock 2023-08-30T19:00:00.000000
on_clock 2023-08-30T20:00:00.000000
on_clock 2023-08-30T21:00:00.000000
on_clock 2023-08-30T22:00:00.000000
on_clock 2023-08-30T23:00:00.000000
on_clock 2023-08-31T00:00:00.000000
on_clock 2023-08-31T01:00:00.000000
on_clock 2023-08-31T02:00:00.000000
on_clock 2023-08-31T03:00:00

In [5]:
session.log_df.sum()

symbol,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,update_time,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change
str,datetime[μs],str,str,str,str,str,f64,f64,f64,str,datetime[μs],f64,f64,f64,f64,str,u32,str,f64,f64,f64,f64,f64,f64,f64,f64
null,null,null,null,null,null,null,0.0,11.88,0.0,null,null,3.1515e7,11.88,315153.8055,0.0,null,0,null,0.0,0.0,-7.6991,0.0,-7.6991,0.0,0.0,0.0
